In [4]:
#Importamos la librería
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
import tqdm
import time

In [5]:
#Definimos el URL
url = 'https://tienda.vivanda.com.pe/tiendas'
page = requests.get(url)

In [6]:
#Estructura bs4
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
#Listamos las Tiendas
td = soup.find_all('h2', class_='fontFamily-Bold color-black')

tiendas = list()


count = 0
for i in td:
    if count<8:
        tiendas.append(i.text)
    else:
        break
    count +=1
print(tiendas)

['Tienda Vivanda Asia ', 'Tienda Vivanda Javier Prado ', 'Tienda Vivanda Pardo ', 'Tienda Vivanda Monterrico ', 'Tienda Vivanda Benavides ', 'Tienda Vivanda Dos de Mayo ', 'Tienda Vivanda Pezet ', 'Tienda Vivanda Libertadores ']


In [8]:
# Listamos las direcciones
def obtener_direccion(url):
    
    response = requests.get(url)
    
    
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
       
        direccion_elemento =pd.Series(soup.find_all('p')[1:])
        
        
        #direccion = direccion_elemento.text if direccion_elemento else "Dirección no encontrada"
        
        return direccion_elemento
    else:
        print("No se pudo acceder a la página:", response.status_code)


url_ejemplo = "https://tienda.vivanda.com.pe/tiendas"
direccion_tienda = obtener_direccion(url_ejemplo)
direccion_tienda

0    [Av. Cayma S/N, KM 97.5 panamerica sur. Refd: ...
1             [Av. Javier Prado Oeste 999, Magdalena.]
2                    [Av. José Pardo 715, Miraflores.]
3    [Av. La Encalada cuadra 5, esquina con av. Ang...
4    [Esquina de la avenida Benavides con la calle ...
5    [Av. Dos de Mayo 1410, esquina con Nogales, Sa...
6    [Av. Juan Antonio Pezet 1340 – 1360, San Isidro.]
7                   [Av. Libertadores 596 San Isidro.]
dtype: object

In [9]:
#Elaboramos un data frame para poder mostrarlo como una tabla
data=pd.concat([pd.Series(tiendas),direccion_tienda],axis=1)
df = pd.DataFrame(data)
df = df.rename(columns={0:"Tienda",1:"Dirección"})
df



,Tienda,Dirección
0,Tienda Vivanda Asia,"[Av. Cayma S/N, KM 97.5 panamerica sur. Refd: ..."
1,Tienda Vivanda Javier Prado,"[Av. Javier Prado Oeste 999, Magdalena.]"
2,Tienda Vivanda Pardo,"[Av. José Pardo 715, Miraflores.]"
3,Tienda Vivanda Monterrico,"[Av. La Encalada cuadra 5, esquina con av. Ang..."
4,Tienda Vivanda Benavides,[Esquina de la avenida Benavides con la calle ...
5,Tienda Vivanda Dos de Mayo,"[Av. Dos de Mayo 1410, esquina con Nogales, Sa..."
6,Tienda Vivanda Pezet,"[Av. Juan Antonio Pezet 1340 – 1360, San Isidro.]"
7,Tienda Vivanda Libertadores,[Av. Libertadores 596 San Isidro.]


In [10]:
for direccion in df["Dirección"]:
    print(direccion)

<p>Av. Cayma S/N, KM 97.5 panamerica sur. Refd: Al lado del grifo Primax, Al lado del Boluevard.</p>
<p>Av. Javier Prado Oeste 999, Magdalena.</p>
<p>Av. José Pardo 715, Miraflores.</p>
<p>Av. La Encalada cuadra 5, esquina con av. Angamos Este cuadra 15, Monterrico.</p>
<p>Esquina de la avenida Benavides con la calle Alcanfores, Miraflores.</p>
<p>Av. Dos de Mayo 1410, esquina con Nogales, San Isidro.</p>
<p>Av. Juan Antonio Pezet 1340 – 1360, San Isidro.</p>
<p>Av. Libertadores 596 San Isidro.</p>


In [11]:
df["Dirección"].astype(str).map(lambda x:x[3:-4])

0    Av. Cayma S/N, KM 97.5 panamerica sur. Refd: A...
1               Av. Javier Prado Oeste 999, Magdalena.
2                      Av. José Pardo 715, Miraflores.
3    Av. La Encalada cuadra 5, esquina con av. Anga...
4    Esquina de la avenida Benavides con la calle A...
5    Av. Dos de Mayo 1410, esquina con Nogales, San...
6      Av. Juan Antonio Pezet 1340 – 1360, San Isidro.
7                     Av. Libertadores 596 San Isidro.
Name: Dirección, dtype: object

In [12]:
for tienda in df["Tienda"]:
    print(tienda)

Tienda Vivanda Asia 
Tienda Vivanda Javier Prado 
Tienda Vivanda Pardo 
Tienda Vivanda Monterrico 
Tienda Vivanda Benavides 
Tienda Vivanda Dos de Mayo 
Tienda Vivanda Pezet 
Tienda Vivanda Libertadores 


In [34]:
df_tienda=df["Tienda"].astype(str).map(lambda x:x[0:-1])
df_tienda

0            Tienda Vivanda Asia
1    Tienda Vivanda Javier Prado
2           Tienda Vivanda Pardo
3      Tienda Vivanda Monterrico
4       Tienda Vivanda Benavides
5     Tienda Vivanda Dos de Mayo
6           Tienda Vivanda Pezet
7    Tienda Vivanda Libertadores
Name: Tienda, dtype: object

In [35]:
def get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService("chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [15]:
list_direcciones_final = []

driver = get_driver()

for  direccion in tqdm.tqdm(df["Dirección"].astype(str).map(lambda x:x[3:-4])):

    dir = direccion + ", Peru"
    #dir = dir.replace('/', ' ').replace('#', ' ').replace('.', ' ')
    
    #driver.get("https://www.google.com/maps/search/"+dir)

    driver.get("https://www.google.com/maps/")

    searchbox = driver.find_element(By.ID, 'searchboxinput')

    searchbox.send_keys(dir)

    searchbutton = driver.find_element(By.ID, 'searchbox-searchbutton')

    searchbutton.click()

    try:
        time.sleep(3)
        coordenadas = driver.current_url
        

        latitud = float(coordenadas[coordenadas.find('@')+1:coordenadas.find(',', coordenadas.find('@')+2)])
        longitud =float(coordenadas[coordenadas.find('-', coordenadas.find('@')+2):coordenadas.find(',', coordenadas.find(',', coordenadas.find('@')+2)+1)])
        
    
        try:
            imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')
        
        except:
            print("Imagen no encontrada")

        nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

        try:
            direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text
            
        except:

            try:
                direccion_maps = driver.find_element(By.CSS_SELECTOR, "span.DkEaL").text
            
            except:
                direccion_maps = ''
                comentario = "Dirección no encontrada"
        
        comentario = 'Dirección encontrada'
    
    except:

        try:
            wait(driver, 4).until(ec.element_to_be_clickable((By.CLASS_NAME, "hfpxzc")))
            boton = driver.find_element(By.CLASS_NAME, "hfpxzc")
            boton.click()

            time.sleep(3)
            coordenadas = driver.current_url

            latitud = float(coordenadas[coordenadas.find('@')+1:coordenadas.find(',', coordenadas.find('@')+2)])
            longitud =float(coordenadas[coordenadas.find('-', coordenadas.find('@')+2):coordenadas.find(',', coordenadas.find(',', coordenadas.find('@')+2)+1)])

            try:
                imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')
            
            except:
                print("Imagen no encontrada")

            nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

            comentario = 'Más de una dirección encontrada, toma la primera'

            try:
                direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text
                
            except:

                try:
                    direccion_maps = driver.find_element(By.CSS_SELECTOR, "span.DkEaL").text
                
                except:
                    direccion_maps = ''
                    comentario = "Dirección no encontrada"
            
            print(comentario)

        except:
            
            comentario = "Dirección no encontrada"
            print(comentario)
        
            latitud = np.nan
            longitud = np.nan

            imagen = ''

            nombre_maps = ''

            direccion_maps = ''
            
            driver.get("https://www.google.com/maps")


    list_direcciones_final.append([direccion,imagen, latitud, longitud, comentario, nombre_maps, direccion_maps])

driver.close()


  0%|          | 0/8 [00:00<?, ?it/s]

Imagen no encontrada


 12%|█▎        | 1/8 [00:11<01:21, 11.62s/it]

Más de una dirección encontrada, toma la primera


 50%|█████     | 4/8 [00:26<00:23,  5.84s/it]

Imagen no encontrada
Dirección no encontrada


100%|██████████| 8/8 [00:50<00:00,  6.25s/it]


In [44]:
list_direcciones_final

[['Av. Cayma S/N, KM 97.5 panamerica sur. Refd: Al lado del grifo Primax, Al lado del Boluevard.',
  'https://lh5.googleusercontent.com/p/AF1QipMlgGEfA9t-4z_bBFiN1suqRbZuLYAte3NR9yYx=w486-h240-k-no',
  -15.8759767,
  -74.17806,
  'Más de una dirección encontrada, toma la primera',
  '1S',
  ''],
 ['Av. Javier Prado Oeste 999, Magdalena.',
  'https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=rdbDchWOXx828CWDhWcneQ&cb_client=search.gws-prod.gps&w=408&h=240&yaw=18.927866&pitch=0&thumbfov=100',
  -12.0931737,
  -77.0604801,
  'Dirección encontrada',
  'Av Javier Prado Oeste 999',
  'Av Javier Prado Oeste 999, Lima 15076'],
 ['Av. José Pardo 715, Miraflores.',
  'https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=H8c1IfoSOn60-f4GTv1kVQ&cb_client=search.gws-prod.gps&w=408&h=240&yaw=213.50693&pitch=0&thumbfov=100',
  -12.1192819,
  -77.0386038,
  'Dirección encontrada',
  'Av. José Pardo 715',
  'Av. José Pardo 715, Miraflores 15074'],
 ['Av. La Encalada cuadra 5, esq

In [45]:
df_tiendas_vivanda = pd.DataFrame(list_direcciones_final, columns=[ 'direccion', 'imagen', 'latitud', 'longitud', 'comentario', 'nombre_maps', 'direccion_maps'])
df_tiendas_vivanda

,direccion,imagen,latitud,longitud,comentario,nombre_maps,direccion_maps
0,"Av. Cayma S/N, KM 97.5 panamerica sur. Refd: A...",https://lh5.googleusercontent.com/p/AF1QipMlgG...,-15.875977,-74.178060,"Más de una dirección encontrada, toma la primera",1S,
1,"Av. Javier Prado Oeste 999, Magdalena.",https://streetviewpixels-pa.googleapis.com/v1/...,-12.093174,-77.060480,Dirección encontrada,Av Javier Prado Oeste 999,"Av Javier Prado Oeste 999, Lima 15076"
2,"Av. José Pardo 715, Miraflores.",https://streetviewpixels-pa.googleapis.com/v1/...,-12.119282,-77.038604,Dirección encontrada,Av. José Pardo 715,"Av. José Pardo 715, Miraflores 15074"
3,"Av. La Encalada cuadra 5, esquina con av. Anga...",https://lh5.googleusercontent.com/p/AF1QipOBeJ...,-12.109180,-76.974744,Dirección encontrada,Vivanda Monterrico,"esquina con, Av. La Encalada cuadra 5, Av. Ang..."
4,Esquina de la avenida Benavides con la calle A...,,NaN,NaN,Dirección no encontrada,,
5,"Av. Dos de Mayo 1410, esquina con Nogales, San...",https://lh5.googleusercontent.com/p/AF1QipOQ7t...,-12.092161,-77.048004,Dirección encontrada,Vivanda,"Av. Dos de Mayo 1410, San Isidro 15073"
6,"Av. Juan Antonio Pezet 1340 – 1360, San Isidro.",https://streetviewpixels-pa.googleapis.com/v1/...,-12.102729,-77.054824,Dirección encontrada,1360,"Av. Gral. Juan Antonio Pezet 1340, San Isidro ..."
7,Av. Libertadores 596 San Isidro.,https://streetviewpixels-pa.googleapis.com/v1/...,-12.103862,-77.040485,Dirección encontrada,C. Los Libertadores 596,"C. Los Libertadores 596, San Isidro 15073"


In [46]:
df_seleccion_Vivanda = df_tiendas_vivanda[['direccion','latitud', 'longitud']]
df_seleccion_Vivanda

,direccion,latitud,longitud
0,"Av. Cayma S/N, KM 97.5 panamerica sur. Refd: A...",-15.875977,-74.178060
1,"Av. Javier Prado Oeste 999, Magdalena.",-12.093174,-77.060480
2,"Av. José Pardo 715, Miraflores.",-12.119282,-77.038604
3,"Av. La Encalada cuadra 5, esquina con av. Anga...",-12.109180,-76.974744
4,Esquina de la avenida Benavides con la calle A...,NaN,NaN
5,"Av. Dos de Mayo 1410, esquina con Nogales, San...",-12.092161,-77.048004
6,"Av. Juan Antonio Pezet 1340 – 1360, San Isidro.",-12.102729,-77.054824
7,Av. Libertadores 596 San Isidro.,-12.103862,-77.040485


In [47]:
df_tienda

0            Tienda Vivanda Asia
1    Tienda Vivanda Javier Prado
2           Tienda Vivanda Pardo
3      Tienda Vivanda Monterrico
4       Tienda Vivanda Benavides
5     Tienda Vivanda Dos de Mayo
6           Tienda Vivanda Pezet
7    Tienda Vivanda Libertadores
Name: Tienda, dtype: object

In [48]:
df['COMPETIDOR']="Vivanda"

In [41]:
resultado = pd.concat([df['COMPETIDOR'],df_tienda,df_seleccion_Vivanda], ignore_index=True,axis=1)
resultado

,0,1,2,3,4
0,Vivanda,Tienda Vivanda Asia,"Av. Cayma S/N, KM 97.5 panamerica sur. Refd: A...",-15.875977,-74.178060
1,Vivanda,Tienda Vivanda Javier Prado,"Av. Javier Prado Oeste 999, Magdalena.",-12.093174,-77.060480
2,Vivanda,Tienda Vivanda Pardo,"Av. José Pardo 715, Miraflores.",-12.119282,-77.038604
3,Vivanda,Tienda Vivanda Monterrico,"Av. La Encalada cuadra 5, esquina con av. Anga...",-12.109180,-76.974744
4,Vivanda,Tienda Vivanda Benavides,Esquina de la avenida Benavides con la calle A...,NaN,NaN
5,Vivanda,Tienda Vivanda Dos de Mayo,"Av. Dos de Mayo 1410, esquina con Nogales, San...",-12.092161,-77.048004
6,Vivanda,Tienda Vivanda Pezet,"Av. Juan Antonio Pezet 1340 – 1360, San Isidro.",-12.102729,-77.054824
7,Vivanda,Tienda Vivanda Libertadores,Av. Libertadores 596 San Isidro.,-12.103862,-77.040485


In [49]:
df_Vivanda = resultado.rename(columns={0:"COMPETIDOR",1:"NOMBRE_LOCAL",2:"DIRECCION",3:"LATITUD",4:"LONGITUD"})
df_Vivanda

,COMPETIDOR,NOMBRE_LOCAL,DIRECCION,LATITUD,LONGITUD
0,Vivanda,Tienda Vivanda Asia,"Av. Cayma S/N, KM 97.5 panamerica sur. Refd: A...",-15.875977,-74.178060
1,Vivanda,Tienda Vivanda Javier Prado,"Av. Javier Prado Oeste 999, Magdalena.",-12.093174,-77.060480
2,Vivanda,Tienda Vivanda Pardo,"Av. José Pardo 715, Miraflores.",-12.119282,-77.038604
3,Vivanda,Tienda Vivanda Monterrico,"Av. La Encalada cuadra 5, esquina con av. Anga...",-12.109180,-76.974744
4,Vivanda,Tienda Vivanda Benavides,Esquina de la avenida Benavides con la calle A...,NaN,NaN
5,Vivanda,Tienda Vivanda Dos de Mayo,"Av. Dos de Mayo 1410, esquina con Nogales, San...",-12.092161,-77.048004
6,Vivanda,Tienda Vivanda Pezet,"Av. Juan Antonio Pezet 1340 – 1360, San Isidro.",-12.102729,-77.054824
7,Vivanda,Tienda Vivanda Libertadores,Av. Libertadores 596 San Isidro.,-12.103862,-77.040485


In [32]:
df_Vivanda.to_csv('Vivanda.csv',index=False)